In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from datetime import datetime
import time
import os
import warnings
warnings.filterwarnings("ignore")

# Associated analysis: C:\Users\A4023862\OneDrive - Astellas Pharma Inc\FPA\Sales Forecasting\FPA Forecasting Data Needs.xlsx

input_folder = r'C:\Users\A4023862\OneDrive - Astellas Pharma Inc\FPA\SKUtoEquivalized'
output_folder =  r'C:\Users\A4023862\OneDrive - Astellas Pharma Inc\FPA\SKUtoEquivalized\outputs'

######################
# READ IN DATA
######################
input_file = 'analysis\Comparison of BPC to IBP v3.xlsx'
path = os.path.join(input_folder, input_file)
# prodv3=pd.read_excel(path, engine='openpyxl', sheet_name = 'prodv3')

maypole_df=pd.read_excel(path, engine='openpyxl', sheet_name = 'maypole_units')

input_file = 'inputs\sku_units_emea_v3.xlsx'
path = os.path.join(input_folder, input_file)
sku1=pd.read_excel(path, engine='openpyxl')

input_file = 'inputs\sku_units_nonemea_v3.xlsx'
path = os.path.join(input_folder, input_file)
sku2=pd.read_excel(path, engine='openpyxl')

input_file = r'mappings\IBP_Maypole_country_mapping_v2.xlsx'
path = os.path.join(input_folder, input_file)
ibpmay_map_p=pd.read_excel(path, engine='openpyxl', sheet_name = 'Products')

ibpmay_map_c=pd.read_excel(path, engine='openpyxl', sheet_name = 'Countries')

input_file = r'conversion_factors\PAM SKU conversion factors vDRAFT 11 NOv+old factors.xlsx'
path = os.path.join(input_folder, input_file)
convfac_map=pd.read_excel(path, sheet_name='PAM SKU values', engine='openpyxl', skiprows = 1)

#####################
# CLEAN MAPPING FILES
#####################
ibpmay_map_c=ibpmay_map_c[['Customer Country', 'Maypole Cost Object']]
ibpmay_map_c.columns = ['region', 'maypole_cost_object']

ibpmay_map_p=ibpmay_map_p[['Product Group (IBP)', 'Maypole mapping', 'Status']]
ibpmay_map_p.columns = ['product', 'maypole_product', 'status']
ibpmay_map_p = ibpmay_map_p[ibpmay_map_p['status']=='Active'].reset_index(drop=True)

In [128]:
######################
# MAYPOLE DATA
######################
maypole=maypole_df

# Replace column names
columns = list(maypole.columns)
columns[0] = 'currency'
columns[1] = 'cost_object'
columns[2] = 'product'
maypole.columns = columns

maypole=maypole.melt(id_vars=["currency", "cost_object", "product"], 
        var_name="ds", 
        value_name="value")

maypole_ds=pd.DataFrame (maypole['ds'].unique(), columns = ['fy_ds'])

maypole_ds['ds'] = ""

# Extract substring of ds to parse into date
for i in range(0,len(maypole_ds)):
    maypole_ds['ds'][i]=maypole_ds['fy_ds'][i].split('- FY',1)[1]

# Convert to date
maypole_ds['ds'] =  pd.to_datetime(maypole_ds['ds'], format='%y %B')
maypole_ds['year'], maypole_ds['month'], maypole_ds['day'] = maypole_ds['ds'].dt.year, maypole_ds['ds'].dt.month, maypole_ds['ds'].dt.day

# Address fiscal year dates
for i in range(0,len(maypole_ds['ds'])):
    if maypole_ds['month'][i] <4:
        maypole_ds['year'][i]=maypole_ds['year'][i]+1
        
# Create new date
maypole_ds['ds']=pd.to_datetime(maypole_ds[['year', 'month', 'day']])

# Merge fiscal year date mapping with original df
maypole=maypole.merge(maypole_ds[['fy_ds', 'ds']], how='left', left_on='ds', right_on='fy_ds')
maypole=maypole[['currency', 'cost_object', 'product', 'ds_y', 'value']]
maypole.columns = ['currency', 'cost_object', 'product', 'ds', 'value']

# Create maypole tag file
maypole_tags = maypole[['cost_object', 'product']]
maypole_tags=maypole_tags.drop_duplicates()
maypole_tags.columns = ['maypole_cost_object', 'maypole_product']
print('Tags in original Maypole template: ')
print(maypole_tags.shape[0])

maypole_tags = maypole_tags[maypole_tags['maypole_product'].isin(ibpmay_map_p['maypole_product'])]
maypole_tags = maypole_tags[maypole_tags['maypole_cost_object'].isin(ibpmay_map_c['maypole_cost_object'])]
print('Tags with "Active" status: ')
print(maypole_tags.shape[0])

# Save maypole data
output_file = r'v2\maypole_df.csv'
path = os.path.join(output_folder, output_file)
maypole.to_csv(path)

maypole

Tags in original Maypole template: 
699
Tags with "Active" status: 
508


,currency,cost_object,product,ds,value
0,LC - LC,D_US - US,P_XTD_TOT - Xtandi Total,2017-04-01,10957.0
1,LC - LC,D_US - US,P_BE_TOT - Myrbetriq/BET Total,2017-04-01,8311680.0
2,LC - LC,D_US - US,P_VC_TOT - Vesicare Total,2017-04-01,7265700.0
3,LC - LC,D_US - US,P_PRG_TOT - Prograf Total,2017-04-01,3565970.0
4,LC - LC,D_US - US,P_ADG_TOT - Advagraf/GRA Total,2017-04-01,265950.0
...,...,...,...,...,...
47527,LC - LC,D_ATW - ATW (Taiwan),P_MYS - Myslee/Stilnox,2022-11-01,-33204557.0
47528,LC - LC,D_ATW - ATW (Taiwan),P_CFIX - Cefspan(Cefixime),2022-11-01,NaN
47529,LC - LC,D_ATW - ATW (Taiwan),P_TR - TR,2022-11-01,NaN
47530,LC - LC,D_ATW - ATW (Taiwan),P_OTHERH2 - Other Astellas products,2022-11-01,NaN


In [129]:
######################
# IBP DATA
######################
sku = pd.concat([sku1, sku2], ignore_index=True)
sku = sku[sku['Key Figure']=='Actual / Ad hoc Sales History']
sku=sku.melt(id_vars=["Product Group", "SC Category ID", "Location ID", "Product ID", "Product Desc", "Customer Country", "Dosage", "Key Figure"], 
        var_name="ds", 
        value_name="value")
# Convert date
sku['ds'] =  pd.to_datetime(sku['ds'], format='%b-%y')

# Convert Product ID to string
sku['Product ID'] = sku['Product ID'].astype(str)


# Calculate old equiv units
sku_new=sku[sku['Product ID'].isin(convfac_map['Mat #'])][['Product Group', 'Customer Country', 'Product Desc', 'Product ID', 'ds', 'value']]
sku_new=sku_new.merge(convfac_map[['Mat #', 'Conversion', 'Old conversion']], how='left', left_on='Product ID', right_on='Mat #')

# Calculate converted values
sku_new['ibp_old']=sku_new['value']*sku_new['Old conversion']
sku_new['ibp_new']=sku_new['value']*sku_new['Conversion']

# Group by product group, customer country, ds
sku_equiv = sku_new.groupby(['Product Group', 'Customer Country', 'ds'])['ibp_old', 'ibp_new'].agg('sum').reset_index()

# Rename IBP columns
sku_equiv.columns = ['product', 'region', 'ds', 'ibp_old', 'ibp_new']

# Convert ibp product and regions to match maypole
sku_equiv=sku_equiv.merge(ibpmay_map_c, how='left', left_on='region', right_on='region')
sku_equiv=sku_equiv.merge(ibpmay_map_p, how='left', left_on='product', right_on='product')

# Convert date to timestamp
sku_equiv['ds']=pd.to_datetime(sku_equiv['ds'])

# Create ibp tag file
ibp_tags = sku_equiv[['product', 'region', 'maypole_cost_object', 'maypole_product']].drop_duplicates()

# Get equiv units by maypole_cost_object, maypole_product, ds
sku_equiv=sku_equiv[['maypole_cost_object', 'maypole_product', 'ds', 'ibp_old', 'ibp_new']]
sku_equiv=sku_equiv.groupby(['maypole_cost_object', 'maypole_product', 'ds'])['ibp_old', 'ibp_new'].sum().reset_index()

sku_equiv

,maypole_cost_object,maypole_product,ds,ibp_old,ibp_new
0,???,P_ADG_TOT - Advagraf/GRA Total,2018-01-01,0.000,310.000000
1,???,P_ADG_TOT - Advagraf/GRA Total,2018-02-01,0.000,0.000000
2,???,P_ADG_TOT - Advagraf/GRA Total,2018-03-01,35.925,35.925000
3,???,P_ADG_TOT - Advagraf/GRA Total,2018-04-01,0.000,391.000000
4,???,P_ADG_TOT - Advagraf/GRA Total,2018-05-01,0.000,221.425000
...,...,...,...,...,...
37205,D_US - US,P_XTD_TOT - Xtandi Total,2022-09-01,27213.000,29156.785714
37206,D_US - US,P_XTD_TOT - Xtandi Total,2022-10-01,26995.000,28923.214286
37207,D_US - US,P_XTD_TOT - Xtandi Total,2022-11-01,30000.000,32142.857143
37208,D_US - US,P_XTD_TOT - Xtandi Total,2022-12-01,30821.000,33022.500000


In [132]:
# Question: Are all the tags in maypole mapped?
maypole_tags2 = maypole_tags.merge(ibp_tags, how='left', on=['maypole_cost_object', 'maypole_product'])
maypole_tags2['tag'] = maypole_tags2['maypole_cost_object'] + maypole_tags2['maypole_product']

print('Tags with "Active" status in Maypole and corresponding IBP match (may not be 1:1): ')
print(maypole_tags2.shape[0])

print('Tags with no corresponding IBP match: ')
print(maypole_tags2[(~maypole_tags2['product'].notnull()) & (~maypole_tags2['region'].notnull())].shape[0])

# Save ibp data
output_file = r'v2\maypole_tags2.csv'
path = os.path.join(output_folder, output_file)
maypole_tags2.to_csv(path)

Tags with "Active" status in Maypole and corresponding IBP match (may not be 1:1): 
781
Tags with no corresponding IBP match: 
13


In [134]:
######################
# MERGE MAYPOLE AND IBP UNITS FOR COMPARISON
######################
comb_df = maypole.merge(sku_equiv[['maypole_cost_object', 'maypole_product', 'ds', 'ibp_old', 'ibp_new']], how='left', left_on=['cost_object','product','ds'], right_on=['maypole_cost_object', 'maypole_product', 'ds'])

# Analysis period
comb_df=comb_df[(comb_df['ds']>='2021-11-01') & (comb_df['ds']<'2022-11-01')]

# Sum values
comb_df2=comb_df.groupby(['cost_object', 'product']).sum().reset_index()
comb_df2['diff%'] = abs((comb_df2['ibp_old']-comb_df2['value']) / comb_df2['value']) * 100

# Create tag
comb_df2['tag'] = comb_df2['cost_object'] + comb_df2['product']

# Filter tags
comb_df2 = comb_df2[comb_df2['tag'].isin(maypole_tags2['tag'])]

print('Tags in Maypole with IBP match: ')
print(comb_df2.shape[0])

print('Tags in Maypole with IBP match with over 10% diff: ')
print(comb_df2[comb_df2['diff%']>10].shape[0])


output_file = r'v2\comb_df2.csv'
path = os.path.join(output_folder, output_file)
comb_df2.to_csv(path)

comb_df2

Tags in Maypole with IBP match: 
508
Tags in Maypole with IBP match with over 10% diff: 
132


,cost_object,product,value,ibp_old,ibp_new,diff%,tag
0,D_ACN - ACN (China),P_ADG_TOT - Advagraf/GRA Total,9.430975e+03,9444.15,9.444150e+03,0.139699,D_ACN - ACN (China)P_ADG_TOT - Advagraf/GRA Total
3,D_ACN - ACN (China),P_BE_TOT - Myrbetriq/BET Total,1.020176e+07,10193070.00,1.019307e+07,0.085181,D_ACN - ACN (China)P_BE_TOT - Myrbetriq/BET Total
4,D_ACN - ACN (China),P_FUN_TOT - Mycamine/FUN Total,3.208730e+05,320550.00,1.602750e+07,0.100663,D_ACN - ACN (China)P_FUN_TOT - Mycamine/FUN Total
6,D_ACN - ACN (China),P_HA_TOT - HA/Tocas/Omnic Total,1.381622e+08,0.00,0.000000e+00,100.000000,D_ACN - ACN (China)P_HA_TOT - HA/Tocas/Omnic T...
7,D_ACN - ACN (China),P_JOSAMYCIN - Josamycin,0.000000e+00,0.00,0.000000e+00,NaN,D_ACN - ACN (China)P_JOSAMYCIN - Josamycin
...,...,...,...,...,...,...,...
694,D_US - US,P_PRG_TOT - Prograf Total,1.785880e+07,17870630.00,1.824980e+04,0.066256,D_US - USP_PRG_TOT - Prograf Total
695,D_US - US,P_TARCEVA_TOT - Tarceva Total,0.000000e+00,0.00,0.000000e+00,NaN,D_US - USP_TARCEVA_TOT - Tarceva Total
696,D_US - US,P_VC_TOT - Vesicare Total,1.171020e+06,1171020.00,1.171020e+06,0.000000,D_US - USP_VC_TOT - Vesicare Total
697,D_US - US,P_XOS_TOT - Xospata total,8.472000e+03,8472.00,9.077143e+03,0.000000,D_US - USP_XOS_TOT - Xospata total


In [123]:
comb_df2[comb_df2['diff%']>10]

,cost_object,product,value,ibp_old,ibp_new,diff%,tag
6,D_ACN - ACN (China),P_HA_TOT - HA/Tocas/Omnic Total,88632740.00,0.00,0.00,100.000000,D_ACN - ACN (China)P_HA_TOT - HA/Tocas/Omnic T...
11,D_ACN - ACN (China),P_TMX - Feburic,13756400.00,0.00,0.00,100.000000,D_ACN - ACN (China)P_TMX - Feburic
13,D_ACN - ACN (China),P_XOS_TOT - Xospata total,218232.00,195090.00,2322.50,10.604311,D_ACN - ACN (China)P_XOS_TOT - Xospata total
14,D_ACN - ACN (China),P_XTD_TOT - Xtandi Total,4120368.00,6870976.00,122696.00,66.756367,D_ACN - ACN (China)P_XTD_TOT - Xtandi Total
21,D_AHK - AHK (Hong Kong),P_TMX - Feburic,2677192.00,0.00,0.00,100.000000,D_AHK - AHK (Hong Kong)P_TMX - Feburic
...,...,...,...,...,...,...,...
655,D_SE - Sweden,P_MDG - Modigraf,30.00,64.00,64.00,113.333333,D_SE - SwedenP_MDG - Modigraf
657,D_SE - Sweden,P_PRG_TOT - Prograf Total,159.35,125.35,125.35,21.336680,D_SE - SwedenP_PRG_TOT - Prograf Total
658,D_SE - Sweden,P_VC_TOT - Vesicare Total,60810.00,54330.00,54330.00,10.656142,D_SE - SwedenP_VC_TOT - Vesicare Total
670,D_SI - Slovenia,P_MDG - Modigraf,3.40,8.24,8.24,142.352941,D_SI - SloveniaP_MDG - Modigraf


In [131]:
max(sku_equiv['ds'])

Timestamp('2023-01-01 00:00:00')